
## Входные данные
Вход читается из файла `in.txt` (в одну строку или через пробелы/переводы строк):

`a b c n [eps] [max_iter]`

- `a` — нижний предел интеграла.
- `b` — правая часть уравнения $\int_a^x f(t)\,dt = b$.
- `c` — правая граница отрезка поиска (используется отрезок $[a,c]$).
- `n` — начальное число разбиений в методе трапеций (для численного интеграла).
- `eps` (необязательно) — требуемая точность по невязке $|\varphi(x)|$ (по умолчанию `1e-6`).
- `max_iter` (необязательно) — максимум итераций Ньютона (по умолчанию `50`).

Примечание: включение/выключение дихотомии задаётся флагом `use_bisection` в `main()`, а не через входные данные.

## Порядок метода трапеций
Метод трапеций имеет **порядок 2**: при шаге сетки $h$ погрешность убывает как $O(h^2)$.
При делении шага пополам (переход от $n$ к $2n$) ошибка должна уменьшаться примерно в 4 раза.

В коде используется оценка Рунге–Ричардсона для трапеций:
$$
\frac{|T_{2n} - T_n|}{2^2 - 1} \approx \text{ошибка},
$$
поэтому критерий остановки записан через множитель $(2^2 - 1)=3$.

## Постановка
Требуется найти $x$, удовлетворяющее уравнению
$$
\int_a^x f(t)\,dt = b.
$$
В программе задаётся функция `f(x)`.

Удобно ввести функцию
$$
\varphi(x) = \int_a^x f(t)\,dt - b,
$$
тогда задача сводится к нахождению корня $\varphi(x)=0$.

## Численное вычисление интеграла
Интеграл $\int_a^x f(t)dt$ вычисляется методом трапеций с `n` разбиениями (`integrate_trap`).

- Если `x > a`, строится равномерная сетка на $[a, x]$.
- Если `x < a`, используется тождество $\int_a^x = -\int_x^a$, чтобы корректно учесть знак.
- Для ускорения при уточнении шага (делении пополам) интеграл не пересчитывается «с нуля», а переиспользуется предыдущая сумма (добавляются только новые точки).

## Метод дихотомии
Чтобы получить надёжное стартовое приближение, можно сначала решить $\varphi(x)=0$ методом дихотомии на отрезке $[a,c]$.

Условие применимости: на концах отрезка должна быть смена знака
$$
\varphi(a)\,\varphi(c) \le 0.
$$
Тогда на каждой итерации берётся середина $m=(l+r)/2$ и по знаку $\varphi(m)$ выбирается подотрезок, на котором остаётся корень.

Важно: в программе на этапе дихотомии интеграл считается грубее, чем в методе Ньютона (иначе будет слишком медленно, потому что дихотомия делает много вычислений $\varphi(x)$).

## Метод Ньютона
Для $\varphi(x)$ выполняется итерационный процесс
$$
x_{k+1} = x_k - \frac{\varphi(x_k)}{\varphi'(x_k)}.
$$
В данной задаче производная выражается просто:
$$
\varphi'(x) = f(x)
$$
(производная интеграла по верхнему пределу).

Значит, шаг Ньютона имеет вид
$$
x_{k+1} = x_k - \frac{\int_a^{x_k} f(t)dt - b}{f(x_k)}.
$$

## Дихотомия -> Ньютон
В текущей версии решения используется схема:
1) дихотомией получаем $x_0 \in [a,c]$;
2) затем уточняем корень методом Ньютона.

Такой подход обычно надёжнее «чистого» Ньютона (Ньютон может уйти далеко при неудачном старте).

## Флаг в `main()`
В `main()` есть переключатель `use_bisection`:
- `True` — сначала дихотомия, потом Ньютон;
- `False` — только Ньютон со стартом `x0 = (a + c)/2`.

## Как работает программа
- Вход: `a b c n [eps] [max_iter]`.
- Если включена дихотомия, старт для Ньютона берётся из `bisection_solve(a, b, a, c, ...)`.
- На каждой итерации Ньютона вычисляется `F = integrate_trap(a, x, ...)` и `phi = F - b`.
- Остановка: `|phi| < eps` или исчерпано `max_iter`.
- Если `f(x) == 0`, шаг Ньютона невозможен (деление на ноль).

## Требования и ограничения
1. **Область определения:** `f(x)` должна быть определена на отрезках интегрирования между `a` и текущим `x`.
2. **Для дихотомии нужна смена знака:** если $\varphi(a)\varphi(c) > 0$, метод дихотомии не применим на $[a,c]$.
3. **Запрет на нулевую производную:** если `f(x_k)=0`, метод Ньютона останавливается.
4. **Согласование точностей:** $\varphi(x)$ считается приближённо (трапеции), поэтому слишком маленький `eps` не даст смысла без достаточной точности интеграла.

## Практические рекомендации
- Подбирайте `c` так, чтобы на $[a,c]$ была смена знака $\varphi$.
- Для повышения точности увеличивают `n` (точность интеграла) и только затем ужесточают `eps`.
